# create dataset

In [ ]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00


In [ ]:
import os
import re
import unicodedata
import pandas as pd
import kagglehub
from sklearn.model_selection import train_test_split
from underthesea import word_tokenize, sent_tokenize
from tqdm import tqdm

# Tải dataset
path = kagglehub.dataset_download("tuannguyenvananh/vietnamese-plain-text-corpus")
print("Dataset path:", path)


# ==== 1. Đọc file văn bản ====
def read_text_files(base_dir):
    data = []
    all_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".txt"):
                all_files.append(os.path.join(root, file))

    print(f"Đang đọc {len(all_files)} file văn bản...")
    for file_path in tqdm(all_files, desc="Đọc file", unit="file"):
        topic = os.path.basename(os.path.dirname(file_path))
        text = None
        for enc in ["utf-8", "utf-16", "utf-8-sig", "latin-1"]:
            try:
                with open(file_path, "r", encoding=enc) as f:
                    text = f.read().strip()
                break
            except Exception:
                continue
        if text:
            data.append({"topic": topic, "text": text})
    return pd.DataFrame(data)


df = read_text_files(path)
print(f"Đã đọc {len(df)} văn bản từ {df['topic'].nunique()} chủ đề.\n")




# ==== 2. Tiền xử lý ====
def normalize_unicode(text):
    return unicodedata.normalize("NFC", text)


def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", " ", text)                 # bỏ URL
    text = re.sub(r"&[a-z]+;", " ", text)               # bỏ mã HTML
    text = re.sub(r"[\U00010000-\U0010ffff]", "", text) # bỏ emoji
    text = re.sub(r"[^0-9a-zA-ZÀ-ỹ\s.,;:?!\"'()%–\-]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def tokenize_vi(text):
    text = normalize_unicode(clean_text(text))
    sentences = sent_tokenize(text)
    tokens = []
    for sent in sentences:
        toks = word_tokenize(sent, format="text")
        tokens.append(f"<s> {toks} </s>")
    return " ".join(tokens)


print("Làm sạch văn bản...")
tqdm.pandas(desc="Clean text")
df["clean_text"] = df["text"].progress_apply(clean_text)

print("Tokenizing văn bản tiếng Việt...")
tqdm.pandas(desc="Tokenize")
df["tokenized"] = df["text"].progress_apply(tokenize_vi)


# ==== 3. Chia tập ====
print("Đang chia dữ liệu thành train/val/test...")
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df["topic"])
train_df, val_df  = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df["topic"])

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}\n")


# ==== 4. Xây dựng corpus ====
def build_corpus(df):
    corpus = []
    for text in tqdm(df["tokenized"], desc="Xây dựng corpus", unit="văn bản"):
        corpus.append("<start_doc>")
        corpus.append(text)
        corpus.append("<end_doc>")
    return " ".join(corpus)


print("Xây dựng tập huấn luyện...")
train_text = build_corpus(train_df)

print("Xây dựng tập validation...")
val_text   = build_corpus(val_df)

print("Xây dựng tập kiểm thử...")
test_text  = build_corpus(test_df)


# ==== 5. Lưu dữ liệu ====
os.makedirs("data_processed", exist_ok=True)
with open("data_processed/train.txt", "w", encoding="utf-8") as f:
    f.write(train_text)
with open("data_processed/val.txt", "w", encoding="utf-8") as f:
    f.write(val_text)
with open("data_processed/test.txt", "w", encoding="utf-8") as f:
    f.write(test_text)

print("Dữ liệu đã được xử lý và lưu vào thư mục ./data_processed")


100%|██████████| 69.6M/69.6M [00:00<00:00, 194MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/tuannguyenvananh/vietnamese-plain-text-corpus/versions/1
Đang đọc 42744 file văn bản...


Đọc file: 100%|██████████| 42744/42744 [00:02<00:00, 17813.44file/s]


Đã đọc 42744 văn bản từ 8 chủ đề.

Làm sạch văn bản...


...

Xây dựng tập validation...


Xây dựng corpus: 100%|██████████| 3847/3847 [00:00<00:00, 504849.27văn bản/s]


Xây dựng tập kiểm thử...


Xây dựng corpus: 100%|██████████| 4275/4275 [00:00<00:00, 602225.08văn bản/s]


Dữ liệu đã được xử lý và lưu vào thư mục ./data_processed
